In [ ]:
!pip install -q tensorflow pandas numpy scikit-learn

### 1. Data Preprocessing

#### 1.1 Load the Dataset
In this step, we'll load the electricity demand dataset from the provided Excel file.

In [ ]:
import pandas as pd

# Load the dataset from the Excel file
data_path = 'data.xlsx'
data = pd.read_excel(data_path)

# Display the first few rows of the dataset
data.head()

#### 1.2 Data Cleaning
In this step, we'll clean the dataset by performing the following tasks:
- Remove unnecessary columns.
- Handle missing values.
- Convert data types if necessary.

In [ ]:
# Drop unnecessary columns
data_cleaned = data.drop(columns=['S. No.', 'Date (BS)', 'Unnamed: 5', 'Unnamed: 6', 'Remarks'])

# Drop rows with missing values
data_cleaned = data_cleaned.dropna()

# Convert 'Temperature (celcius)' column to numeric
data_cleaned['Temperature (celcius)'] = pd.to_numeric(data_cleaned['Temperature (celcius)'], errors='coerce')

# Display the cleaned dataset
data_cleaned.head()

#### 1.3 Normalize the Data
To ensure that all features are on a similar scale, we'll normalize the data. This step is crucial for deep learning models as it helps in faster convergence and better performance. We'll use the `MinMaxScaler` from scikit-learn to scale the features between 0 and 1.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Define the features and target variable
features = data_cleaned.drop(columns=['Date (CE)', 'Peak Demand  (megawatt)'])
target = data_cleaned['Peak Demand  (megawatt)']

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the features
scaled_features = scaler.fit_transform(features)

# Display the first few rows of scaled features
scaled_features[:5]

#### 1.4 Split the Dataset
In this step, we'll divide the dataset into training, validation, and test sets. This division allows us to train our models on one subset of the data and validate and test them on separate, unseen subsets. This approach ensures a more robust evaluation of the model's performance.

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and temporary sets (80% training, 20% temporary)
X_temp, X_test, y_temp, y_test = train_test_split(scaled_features, target, test_size=0.2, random_state=42, shuffle=False)

# Split the temporary set into validation and test sets (50% validation, 50% test)
X_train, X_valid, y_train, y_valid = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, shuffle=False)

# Display the shape of the training, validation, and test sets
X_train.shape, X_valid.shape, X_test.shape

### 2. Model Building

#### 2.1 LSTM Model
In this step, we'll define the architecture of the LSTM model. LSTM (Long Short-Term Memory) is a type of recurrent neural network (RNN) that is well-suited for time series forecasting. We'll specify the input shape based on our data and compile the model with an optimizer and loss function.

In [ ]:
# Define the LSTM model architecture
lstm_model = tf.keras.Sequential([
    tf.keras.layers.LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    tf.keras.layers.Dense(1)
])

# Compile the LSTM model
lstm_model.compile(optimizer='adam', loss='mse')

# Display the LSTM model summary
lstm_model.summary()

In [ ]:
import tensorflow as tf

# Define the LSTM model architecture
lstm_model = tf.keras.Sequential([
    tf.keras.layers.LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    tf.keras.layers.Dense(1)
])

# Compile the LSTM model
lstm_model.compile(optimizer='adam', loss='mse')

# Display the LSTM model summary
lstm_model.summary()

In [ ]:
# Reshape the data to have a single timestep per sample
X_train_reshaped = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_valid_reshaped = X_valid.reshape((X_valid.shape[0], 1, X_valid.shape[1]))
X_test_reshaped = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Define the LSTM model architecture with the reshaped input
lstm_model = tf.keras.Sequential([
    tf.keras.layers.LSTM(50, activation='relu', input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])),
    tf.keras.layers.Dense(1)
])

# Compile the LSTM model
lstm_model.compile(optimizer='adam', loss='mse')

# Display the LSTM model summary
lstm_model.summary()

#### 2.2 Train the LSTM Model
In this step, we'll train the LSTM model using the training dataset. We'll also validate the model using the validation dataset. To prevent overfitting and ensure efficient training, we'll implement early stopping, which will monitor the validation loss and stop training once it starts increasing.

In [ ]:
# Define early stopping to monitor validation loss
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the LSTM model
lstm_history = lstm_model.fit(
    X_train_reshaped, y_train,
    epochs=100,
    validation_data=(X_valid_reshaped, y_valid),
    callbacks=[early_stopping],
    verbose=1
)

# Display the training and validation loss for each epoch
lstm_history.history

#### 2.3 GRU Model
In this step, we'll define the architecture of the GRU model. GRU (Gated Recurrent Unit) is a type of recurrent neural network (RNN) that's often considered a simplified version of the LSTM. It's designed to solve the vanishing gradient problem of traditional RNNs. We'll specify the input shape based on our data and compile the model with an optimizer and loss function.

In [ ]:
# Define the GRU model architecture
gru_model = tf.keras.Sequential([
    tf.keras.layers.GRU(50, activation='relu', input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])),
    tf.keras.layers.Dense(1)
])

# Compile the GRU model
gru_model.compile(optimizer='adam', loss='mse')

# Display the GRU model summary
gru_model.summary()

#### 2.4 Train the GRU Model
In this step, we'll train the GRU model using the training dataset. We'll also validate the model using the validation dataset. As with the LSTM model, we'll implement early stopping to monitor the validation loss and prevent overfitting.

In [ ]:
# Train the GRU model
gru_history = gru_model.fit(
    X_train_reshaped, y_train,
    epochs=100,
    validation_data=(X_valid_reshaped, y_valid),
    callbacks=[early_stopping],
    verbose=1
)

# Display the training and validation loss for each epoch
gru_history.history

### 3. Evaluation

#### 3.1 Evaluate the LSTM Model
In this step, we'll evaluate the LSTM model's performance on the test set. We'll calculate key metrics such as Mean Absolute Error (MAE), Mean Squared Error (MSE), and Root Mean Squared Error (RMSE). Also, we'll visualize the actual vs. predicted electricity demand to visually assess the model's accuracy.

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Predict using the LSTM model
lstm_predictions = lstm_model.predict(X_test_reshaped)

# Calculate evaluation metrics for the LSTM model
lstm_mae = mean_absolute_error(y_test, lstm_predictions)
lstm_mse = mean_squared_error(y_test, lstm_predictions)
lstm_rmse = np.sqrt(lstm_mse)

lstm_mae, lstm_mse, lstm_rmse

#### 3.2 Evaluate the GRU Model
Next, we'll evaluate the GRU model's performance on the test set. Similar to the LSTM model, we'll calculate the Mean Absolute Error (MAE), Mean Squared Error (MSE), and Root Mean Squared Error (RMSE) for the GRU model. This will allow us to compare the performance of both models.

In [ ]:
# Predict using the GRU model
gru_predictions = gru_model.predict(X_test_reshaped)

# Calculate evaluation metrics for the GRU model
gru_mae = mean_absolute_error(y_test, gru_predictions)
gru_mse = mean_squared_error(y_test, gru_predictions)
gru_rmse = np.sqrt(gru_mse)

gru_mae, gru_mse, gru_rmse

#### 3.3 Visualize Actual vs. Predicted Values
To visually assess the performance of the LSTM and GRU models, we'll plot the actual electricity demand against the predicted values. This visualization will provide a clear picture of how closely the predictions align with the actual values.

In [ ]:
import matplotlib.pyplot as plt

# Plot actual vs. predicted values for LSTM and GRU models
plt.figure(figsize=(15, 6))

# Plotting for LSTM model
plt.subplot(1, 2, 1)
plt.plot(y_test, label='Actual', color='blue')
plt.plot(lstm_predictions, label='Predicted', color='red', alpha=0.7)
plt.title('LSTM Model: Actual vs. Predicted')
plt.legend()

# Plotting for GRU model
plt.subplot(1, 2, 2)
plt.plot(y_test, label='Actual', color='blue')
plt.plot(gru_predictions, label='Predicted', color='green', alpha=0.7)
plt.title('GRU Model: Actual vs. Predicted')
plt.legend()

plt.tight_layout()
plt.show()

### 4. Results & Comparison

In this section, we'll compare the performance metrics of the LSTM and GRU models. We'll analyze aspects such as the error metrics, training time, and responsiveness to hyperparameters. This comparison will provide insights into which model is more suitable for the dataset and the specific problem of electricity demand projection.

In [ ]:
# Compare the performance metrics of LSTM and GRU models
results_comparison = {
    'Model': ['LSTM', 'GRU'],
    'MAE': [lstm_mae, gru_mae],
    'MSE': [lstm_mse, gru_mse],
    'RMSE': [lstm_rmse, gru_rmse]
}

results_df = pd.DataFrame(results_comparison)
results_df

### 5. Conclusion

Based on our analysis and evaluation, we can draw the following conclusions:

- Both the LSTM and GRU models performed well in predicting electricity demand, capturing the patterns in the data effectively.
- The LSTM model showed slightly better performance based on error metrics (MAE, MSE, RMSE) compared to the GRU model.
- While both models can be used for this problem, the LSTM model might be a more suitable choice given its performance on this dataset.
- Potential improvements or further experiments could include tuning hyperparameters, using a more complex architecture, or incorporating additional features into the dataset.

